In [1]:
import pandas as pd
import re  # For preprocessing
import pandas as pd  # For data handling
import spacy  # For preprocessing
import numpy as np
from langdetect import detect
import swifter 

In [2]:
path = '/Users/shafz/OneDrive/Documents/deep-learning-final-project-yelp_reviews_classification/data/raw/'
reviews = pd.read_csv(path+'raw_reviews.csv')
reviews.drop('Unnamed: 0', axis =1, inplace = True)
reviews.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [3]:
path = '/Users/shafz/OneDrive/Documents/deep-learning-final-project-yelp_reviews_classification/data/raw/'
business = pd.read_csv(path+'raw_business.csv')

restaurants_ids = business[business.categories.fillna('-').str.lower().str.contains('restaurant')]
restaurants = reviews[reviews['business_id'].isin(restaurants_ids['business_id'].tolist())]

In [4]:
restaurants.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31


In [6]:
restaurants.shape

(4724684, 9)

In [7]:
restaurants['business_id'].value_counts()

_ab50qdWOk0DdB6XOrBitw    7673
ac1AeYqs8Z4_e2X5M3if2A    7516
GXFMD0Z4jEVZBCsbPf4CTQ    6160
ytynqOUb3hjKeJfRj5Tshw    5778
oBNrLz4EDhiscSlbOl8uAw    5264
                          ... 
NJt2qvQ9NvB7uCxYUnW-dQ       5
7_erbDC2n4I6Zg_e25wcIg       5
DZ7EzF2c6OaNmZ5nXit1MQ       5
Yl4Am6NhqWo_YMmo1SyU5A       5
Tj9FmBCHd84kjAE9vcoBnw       5
Name: business_id, Length: 52286, dtype: int64

In [8]:
restaurants_filtered = restaurants.groupby('business_id').filter(lambda x : len(x)>1000)

In [9]:
restaurants_filtered.shape

(569035, 9)

In [10]:
restaurants_filtered = restaurants_filtered[['business_id', 'stars', 'text']]

# Data Cleaning 
Lets take a look at some reviews to see what data cleaning measures we need 
* We can see that we need to remove punctuation and that some reviews have a lot of white space 
* Both uppercase and lowercase letters are present 
* There are numerical digits as well 
* Some reviews contain expressions such as "this/that" and simply removing '/' results in "thisthat" instead of "this that" 
* Some reviews contained only punctuation (e.g. '.', ': )', '*') 
* Some reviews do not have proper spacing (e.g. 'Don't misinterpret my review....I' which turns into 'dont misinterpret my reviewi')
* There are reviews in Chinese and Spanish 
* Had some strange letters show up such as 'entrÃ©es' due to encoding/decoding issues
* Some reviews have website links in them 

In [11]:
for index,text in enumerate(restaurants_filtered['text'][-5:]):
  print('Review %d:\n'%(index+1),text)

Review 1:
 Please come hungry!!!! Every time I've been in I've never been disappointed. The mimosas are perfect and I can never seem to order just one. My go to are the eggs Benedict's served on soft fluffy biscuits. The portions are generous and filling.
Review 2:
 I've been to the other Federal Donuts location multiple times. I love donuts, and coffee and this place does both very well. The South Philly location is so small, and I often find myself cramped while in there. However, the donuts are always amazing, which keeps me coming back for more and more. I recently visited the Center City location - and found that they had a better handle on crowd control, which I appreciated. The donuts were just as good as what I had tried in the past - specifically, I had the green tea sesame which I could have bathed in vats of. I ate all the little crumbly pieces that had fallen off into the bag when I got home. That's how good they are. Both hot and fancy, really in any variety are always ama

In [12]:
def detect_language(text):
    try:
        lang = detect(text)
    except:
        lang = 'unknown'
    return lang


In [13]:
restaurants_filtered['language'] = restaurants_filtered['text'].swifter.apply(detect_language)

Pandas Apply: 100%|██████████| 569035/569035 [1:36:38<00:00, 98.14it/s]  


In [15]:
restaurants_filtered.loc[restaurants_filtered['language'] != 'en']

,business_id,stars,text,language
3369,jMZ56S8Y1t7cA1Ob-d-qeA,5,Op suggestie van yelp- hier geweest op onze ee...,nl
10274,8uF-bhJFgT4Tn6DTb27viA,5,O05 hj 0fp Ncc83bb0 hk \nj6BChbhnnp phmyj9g5 6...,vi
12471,g04aAvgol7IW8buqSbT4xA,4,Ideal la forma como la espera a las afueras de...,es
18682,GBTPC53ZrG1ZBY3DT8Mbcw,5,P ml hi o Jv 89274077147 mm \np I'm I l l eat ...,cs
35804,SZU9c8V2GuREDN5KgyHFJw,5,"Eu daria 10 estrelas ,se fosse possível!\nUm r...",pt
...,...,...,...,...
6915959,JHRlwxxKY0JJcU97rJ-Bug,5,"Good food, better drinks, best dancing",da
6945551,rgeuy1qbw6Z8B6CSVANHIA,4,"Muy buenas las donas de Federal Donuts, un muy...",es
6949896,skY6r8WAkYqpV7_TxNm23w,5,Wow quel vue quel moment magique à voir la nou...,fr
6961575,QHWYlmVbLC3K6eglWoHVvA,5,"Rico , rico !! Buen servicio, nice place , con...",es


In [16]:
restaurants_english = restaurants_filtered[restaurants_filtered['language'] == 'en']

In [18]:
restaurants_english.shape
# 569035

(568227, 4)

In [19]:
## Expanding Contractions 
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))


def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

In [56]:
# expanding contractions 
restaurants_english['cleaned'] = restaurants_english['text'].apply(lambda x:expand_contractions(x))

Pandas Apply: 100%|██████████| 568227/568227 [01:13<00:00, 7740.09it/s]
<ipython-input-56-867f182548ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_english['cleaned'] = restaurants_english['text'].swifter.apply(lambda x:expand_contractions(x))


In [57]:
## Making all words lowercase, removing punctuation, URLs, and white spaces 
restaurants_english['cleaned'] = restaurants_english['cleaned'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', ' ', x))
restaurants_english['cleaned'] = restaurants_english['cleaned'].str.lower().apply(lambda x: re.sub(r"[\d\n\-\./]+", ' ', x))
restaurants_english['cleaned'] = restaurants_english['cleaned'].apply(lambda x: re.sub(' +',' ',x))
restaurants_english['cleaned'] = restaurants_english['cleaned'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


<ipython-input-57-ce780160fa92>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_english['cleaned'] = restaurants_english['cleaned'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', ' ', x))
<ipython-input-57-ce780160fa92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_english['cleaned'] = restaurants_english['cleaned'].str.lower().apply(lambda x: re.sub(r"[\d\n\-\./]+", ' ', x))
<ipython-input-57-ce780160fa92>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [58]:
restaurants_english['cleaned'] = restaurants_english['cleaned'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')

<ipython-input-58-e862f5b2c85c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_english['cleaned'] = restaurants_english['cleaned'].str.normalize('NFKD')\


In [60]:
restaurants_english['cleaned'] = restaurants_english['cleaned'].replace(' ', np.nan)

<ipython-input-60-d0a2534b201a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_english['cleaned'] = restaurants_english['cleaned'].replace(' ', np.nan)


In [61]:
restaurants_english.isna().sum()

business_id    0
stars          0
text           0
cleaned        1
lemmatized     0
dtype: int64

In [62]:
restaurants_english.dropna(inplace=True)

c:\Users\shafz\anaconda3\envs\practice\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [63]:
for index,text in enumerate(restaurants_english['cleaned'][-5:]):
  print('Review %d:\n'%(index+1),text)


Review 1:
 please come hungry every time i have been in i have never been disappointed the mimosas are perfect and i can never seem to order just one my go to are the eggs benedict is served on soft fluffy biscuits the portions are generous and filling 
Review 2:
 i have been to the other federal donuts location multiple times i love donuts and coffee and this place does both very well the south philly location is so small and i often find myself cramped while in there however the donuts are always amazing which keeps me coming back for more and more i recently visited the center city location and found that they had a better handle on crowd control which i appreciated the donuts were just as good as what i had tried in the past specifically i had the green tea sesame which i could have bathed in vats of i ate all the little crumbly pieces that had fallen off into the bag when i got home that is how good they are both hot and fancy really in any variety are always amazing try them all


In [66]:
restaurants_english.drop('language', axis = 1, inplace=True)


c:\Users\shafz\anaconda3\envs\practice\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [67]:
restaurants_english.head()

,business_id,stars,text,cleaned
11,EQ-TZ2eeD_E0BHuvoaeG5Q,4,"Locals recommended Milktooth, and it's an amaz...",locals recommended milktooth and it is an amaz...
27,S2Ho8yLxhKAa26pBAm6rxA,3,"Service was crappy, and food was mediocre. I ...",service was crappy and food was mediocre i wis...
29,ltBBYdNzkeKdCNPDAsxwAA,2,I at least have to give this restaurant two st...,i at least have to give this restaurant two st...
39,Zx7n8mdt8OzLRXVzolXNhQ,5,Amazing biscuits and (fill in the blank). Grea...,amazing biscuits and fill in the blank great c...
44,W4ZEKkva9HpAdZG88juwyQ,3,"In a word... ""OVERRATED!"". The food took fore...",in a word overrated the food took forever to c...


### Spell Check

In [18]:
from symspellpy import SymSpell, Verbosity
import pkg_resources

In [5]:
restaurants_english = pd.read_csv('./reviews_cleaned.csv')
#reviews.drop('Unnamed: 0', axis =1, inplace = True)
restaurants_english.head(5)

,business_id,stars,text,cleaned
0,EQ-TZ2eeD_E0BHuvoaeG5Q,4,"Locals recommended Milktooth, and it's an amaz...",locals recommended milktooth and it is an amaz...
1,S2Ho8yLxhKAa26pBAm6rxA,3,"Service was crappy, and food was mediocre. I ...",service was crappy and food was mediocre i wis...
2,ltBBYdNzkeKdCNPDAsxwAA,2,I at least have to give this restaurant two st...,i at least have to give this restaurant two st...
3,Zx7n8mdt8OzLRXVzolXNhQ,5,Amazing biscuits and (fill in the blank). Grea...,amazing biscuits and fill in the blank great c...
4,W4ZEKkva9HpAdZG88juwyQ,3,"In a word... ""OVERRATED!"". The food took fore...",in a word overrated the food took forever to c...


In [19]:
# load a dictionary (this one consists of 82,765 English words)
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [38]:
custom_words = [('waitlist', 100)]
for word, freq in custom_words:
    sym_spell.create_dictionary_entry(word, freq)

In [39]:
def spell_check_text(text):
    words = text.split()
    checked_words = []
    for word in words:
        # Check if word is in dictionary
        if sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=0):
            checked_words.append(word)
        else:
            # Get suggestions for misspelled word
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
            if len(suggestions) > 0:
                checked_words.append(suggestions[0].term)
            else:
                checked_words.append(' ')
    return ' '.join(checked_words)

In [40]:
# Apply spell_check_text function to text column of DataFrame
restaurants_english['spell_checked'] = restaurants_english['cleaned'].swifter.apply(spell_check_text)

Pandas Apply: 100%|██████████| 568226/568226 [05:11<00:00, 1822.85it/s]


In [44]:
restaurants_english.head(10)

,business_id,stars,text,cleaned,spell_checked
0,EQ-TZ2eeD_E0BHuvoaeG5Q,4,"Locals recommended Milktooth, and it's an amaz...",locals recommended milktooth and it is an amaz...,locals recommended and it is an amazing jewe...
1,S2Ho8yLxhKAa26pBAm6rxA,3,"Service was crappy, and food was mediocre. I ...",service was crappy and food was mediocre i wis...,service was crappy and food was mediocre i wis...
2,ltBBYdNzkeKdCNPDAsxwAA,2,I at least have to give this restaurant two st...,i at least have to give this restaurant two st...,i at least have to give this restaurant two st...
3,Zx7n8mdt8OzLRXVzolXNhQ,5,Amazing biscuits and (fill in the blank). Grea...,amazing biscuits and fill in the blank great c...,amazing biscuits and fill in the blank great c...
4,W4ZEKkva9HpAdZG88juwyQ,3,"In a word... ""OVERRATED!"". The food took fore...",in a word overrated the food took forever to c...,in a word overrated the food took forever to c...
5,I6L0Zxi5Ww0zEWSAVgngeQ,4,The cafe was extremely cute. We came at 8am an...,the cafe was extremely cute we came at am and ...,the cafe was extremely cute we came at am and ...
6,EtKSTHV5Qx_Q7Aur9o4kQQ,5,"On a scale of one to things that are awesome, ...",on a scale of one to things that are awesome t...,on a scale of one to things that are awesome t...
7,SZU9c8V2GuREDN5KgyHFJw,5,We were a bit weary about trying the Shellfish...,we were a bit weary about trying the shellfish...,we were a bit weary about trying the shellfish...
8,oQ5CPRt0R3AzFvcjNOqB1w,3,Honestly the food doesn't knock my socks off b...,honestly the food does not knock my socks off ...,honestly the food does not knock my socks off ...
9,Zi-F-YvyVOK0k5QD7lrLOg,5,Definitely recommend for anyone looking for a ...,definitely recommend for anyone looking for a ...,definitely recommend for anyone looking for a ...


In [45]:
restaurants_english['spell_checked'] = restaurants_english['spell_checked'].apply(lambda x: re.sub(' +',' ',x))

In [46]:
restaurants_english.to_csv('./reviews_cleaned.csv', index=False)

Removing stop words, lemmatizing, and tokenizing 


In [47]:
#nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

In [48]:
print(nlp.Defaults.stop_words)
len(nlp.Defaults.stop_words)

{'therein', 'show', 'hers', 'wherein', 'most', 'call', 'before', 'latterly', 'yourself', 'any', 'her', '‘s', 'who', 'rather', 'well', 'sixty', 'these', 'yourselves', 'more', 'whereas', 'own', 'around', 'into', 'say', 'him', 'eight', 'must', 'latter', 'fifteen', 'fifty', 'whenever', 'otherwise', 'along', 'nowhere', 'thence', 'whose', 'have', 'put', 'as', 'mine', 'seeming', 'too', 'nobody', 'whereby', 'does', 'elsewhere', 'how', 'make', 'whither', 'nothing', 'had', 'least', 'get', 'may', 'none', 'about', 'further', 'meanwhile', 'where', 'was', 'themselves', 'here', 'across', 'anyway', 'if', 'often', "'ll", '‘d', 'together', 'bottom', 'himself', 'go', 'top', 'beside', 'us', 'towards', 'anywhere', 'anyone', 'amongst', 'you', 'full', 'please', '’m', 'either', 'therefore', 'through', 'so', 'not', 'on', 'mostly', 'become', 'below', 'against', 'few', 'via', 'wherever', 'former', 'such', 'twenty', 'above', 'should', 'within', 'am', 'really', 'seem', 'moreover', 'whatever', 'no', 'still', 'once'

326

In [49]:
nlp.Defaults.stop_words -= {'few', 'whereas', 'unless', 'anything', 'everything', 'nothing','but', 'before', 'afterwards', 'besides',  'anywhere', 'everyone', 'however', 'against', 'never', 'cannot', 'even', 'neither', 'empty', 'nor','well', 'either', 'least', 'less', 'none', 'side', 'not', 
  'full'}
len(nlp.Defaults.stop_words)

298

In [50]:
nlp.Defaults.stop_words |= {'food', 'service', 'drinks', 'restaurant', 'come', 'place', 'get', 'go'}
len(nlp.Defaults.stop_words)

304

In [51]:
texts = restaurants_english['cleaned'].tolist()
lemmatized_texts = []
for doc in nlp.pipe(texts, batch_size=1000, n_process=4):
    lemmatized_texts.append(' '.join([token.lemma_ for token in doc if (token.is_stop==False)]))

In [54]:
restaurants_english['lemmatized'] = lemmatized_texts

In [55]:
restaurants_english.head(10)

,business_id,stars,text,cleaned,spell_checked,lemmatized
0,EQ-TZ2eeD_E0BHuvoaeG5Q,4,"Locals recommended Milktooth, and it's an amaz...",locals recommended milktooth and it is an amaz...,locals recommended and it is an amazing jewel ...,local recommend milktooth amazing jewel indian...
1,S2Ho8yLxhKAa26pBAm6rxA,3,"Service was crappy, and food was mediocre. I ...",service was crappy and food was mediocre i wis...,service was crappy and food was mediocre i wis...,crappy mediocre wish pick dinner town
2,ltBBYdNzkeKdCNPDAsxwAA,2,I at least have to give this restaurant two st...,i at least have to give this restaurant two st...,i at least have to give this restaurant two st...,least star decent but dinner meeting spend e...
3,Zx7n8mdt8OzLRXVzolXNhQ,5,Amazing biscuits and (fill in the blank). Grea...,amazing biscuits and fill in the blank great c...,amazing biscuits and fill in the blank great c...,amazing biscuit fill blank great cocktail high...
4,W4ZEKkva9HpAdZG88juwyQ,3,"In a word... ""OVERRATED!"". The food took fore...",in a word overrated the food took forever to c...,in a word overrated the food took forever to c...,word overrate take forever burger way overcook...
5,I6L0Zxi5Ww0zEWSAVgngeQ,4,The cafe was extremely cute. We came at 8am an...,the cafe was extremely cute we came at am and ...,the cafe was extremely cute we came at am and ...,cafe extremely cute come even jazz band play t...
6,EtKSTHV5Qx_Q7Aur9o4kQQ,5,"On a scale of one to things that are awesome, ...",on a scale of one to things that are awesome t...,on a scale of one to things that are awesome t...,scale thing awesome bomb draw promise duckfat ...
7,SZU9c8V2GuREDN5KgyHFJw,5,We were a bit weary about trying the Shellfish...,we were a bit weary about trying the shellfish...,we were a bit weary about trying the shellfish...,bit weary try shellfish company wharf place li...
8,oQ5CPRt0R3AzFvcjNOqB1w,3,Honestly the food doesn't knock my socks off b...,honestly the food does not knock my socks off ...,honestly the food does not knock my socks off ...,honestly knock sock but people love husband li...
9,Zi-F-YvyVOK0k5QD7lrLOg,5,Definitely recommend for anyone looking for a ...,definitely recommend for anyone looking for a ...,definitely recommend for anyone looking for a ...,definitely recommend look delicious meal reaso...


In [56]:
restaurants_english.to_csv('./lemmatized_reviews.csv')